In [66]:
import lightgbm as lgb
import pandas as pd
import pickle
import numpy as np
import gc
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [67]:
# project_root_path = '/content/drive/MyDrive/NCCU1102/WSM/proj3/RecSys-Dressipi/'
project_root_path = '../..'
n_train_sample = 500

In [95]:
def select_features( X ):
    select_cols = [1,2,3,4,5,6,7,8,9]
    new_X = np.zeros((len(X), len(select_cols)))
    for i in range(len(X)):
         new_X[i] = X[i][select_cols]
        
    return new_X

        

In [96]:
def train_model( i ):
    # fit training data
    X_path = project_root_path + '/dataset/train_features/train_X_' + str(i) + '.pickle'
    y_path = project_root_path + '/dataset/train_features/train_y_' + str(i) + '.pickle'
    with open( X_path, 'rb') as f:
        X = pickle.load(f)
    with open( y_path, 'rb') as f:
        y = pickle.load(f)

    query_train = [n_train_sample] * (len(X)//n_train_sample)

    print("Starting training... train_" + str(i) )
    start_time = time.time()
    gbm = lgb.LGBMRanker(device = 'cpu')
    if i == 0:
        gbm.fit(select_features(X), y, group=query_train)
    else:
        gbm.fit(select_features(X), y, group=query_train, init_model='../../model/lgbm/lgbm_' + str(i-1))
    gbm.booster_.save_model('../../model/lgbm/lgbm_' + str(i))
    print("Training finished " + str(time.time() - start_time))
    
    return gbm


In [97]:
def MRR( X_sessions, y_sessions, y_preds ):
    score = []
    for X, y, pred in zip( X_sessions, y_sessions, y_preds ):
        top100_index = np.argsort(pred)[-100:]
        rank_result = []
        flag = False
        for count, index in enumerate(top100_index[::-1]):
            if y[index] == 1:
                score.append(1/(count+1))
                flag = True
                break
        if not flag:
            score.append(0)
    return np.mean(score)

def pred2rank_result( session_id, X, pred ):
    top100_index = np.argsort(pred)[-100:]
    rank_result = []
    for count, index in enumerate(top100_index[::-1]):
        row = ','.join([str(session_id), str(int(X[index][0])), str(count+1)])
        rank_result.append(row)
    return rank_result

def predict_session( gbm, session_id ):
    with open('../../dataset/test_features/test_X_' + str(session_id) + '.pickle', 'rb') as f:
        X = pickle.load(f)
    pred = gbm.predict(select_features(X))
    return pred2rank_result( session_id, X, pred )



## Train

In [98]:
for i in range(9):
    model = train_model( i )

Starting training... train_0
Training finished 15.399232625961304
Starting training... train_1
Training finished 14.08199167251587
Starting training... train_2
Training finished 14.895485639572144
Starting training... train_3
Training finished 17.14350414276123
Starting training... train_4
Training finished 17.757127285003662
Starting training... train_5
Training finished 19.50046944618225
Starting training... train_6
Training finished 20.91445803642273
Starting training... train_7
Training finished 23.590585947036743
Starting training... train_8
Training finished 4.327640056610107


## Validate

In [20]:
%%time

bst = lgb.Booster(model_file='../../model/lgbm/lgbm_31')
for validate_id in range(8,10):
    # fit training data
    X_path = project_root_path + '/dataset/train_features/train_X_' + str(validate_id) + '.pickle'
    y_path = project_root_path + '/dataset/train_features/train_y_' + str(validate_id) + '.pickle'
    with open( X_path, 'rb') as f:
        X = pickle.load(f)
    with open( y_path, 'rb') as f:
        y = pickle.load(f)

    X_sessions = np.array_split(X, len(X)//n_train_sample)
    y_sessions = np.array_split(y, len(y)//n_train_sample)

    del X
    del y
    gc.collect()

    y_preds = [bst.predict(select_features(X_session)) for X_session in X_sessions]
    print(MRR( X_sessions, y_sessions, y_preds))
    del X_sessions
    del y_sessions
    del y_preds
    gc.collect()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

0.46994645260165346
0.4689435812922673
CPU times: user 42min 2s, sys: 4.21 s, total: 42min 6s
Wall time: 3min 55s


## Predict

In [99]:
bst = lgb.Booster(model_file='../../model/lgbm/lgbm_8')
bst

In [100]:
results = []
with open('../../dataset/test_leaderboard_uniq_sessions') as f:
    for line in tqdm(f.readlines()[1:]):
        session_id = int(line) 
        results.extend(predict_session(bst, session_id ))

100%|██████████| 50000/50000 [03:15<00:00, 255.59it/s]


In [101]:
with open('../../result/lgbm_s500_2021-5_f1-9.csv', 'w') as f:
    f.write('session_id,item_id,rank\n')
    f.write('\n'.join(results))

In [148]:
popu_df = pd.read_csv('../../dataset/uniq_item_id.csv', 'r', index_col='item_id')
date_range = ["2020-01","2020-02","2020-03","2020-04","2020-05","2020-06","2020-07","2020-08","2020-09","2020-10","2020-11","2020-12","2021-01","2021-02","2021-03","2021-04","2021-05","2021-06"]
for col in date_range:
    popu_df[col] = 0

/home/yhl/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3398: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [149]:
with open('../../dataset/train_sessions.csv', 'r') as f:
    for row in f.readlines()[1:]:
        item_id = int(row.split(',')[1])
        date = row.split(',')[2][:7]
        popu_df.loc[item_id, date] += 1
popu_df

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06
item_id,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,16,22,18,51,49,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95,190,90,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,27,17,15,15,53,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28139,0,0,0,0,15,130,77,92,2,0,0,0,0,0,0,0,0,0
28140,0,0,0,0,0,0,0,0,0,152,136,87,53,20,0,1,0,0
28141,0,0,0,0,0,0,0,0,0,53,32,26,7,20,10,4,0,0


In [151]:
popu_df.to_csv('../VSM/popular_train_can.csv')